In [ ]:
%pip install -q selenium webdriver_manager

## 한번만 실행

In [11]:
import subprocess
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pickle

# Chrome 브라우저를 원격 디버깅 모드로 실행
chrome_path = "C:/Program Files/Google/Chrome/Application/chrome.exe"
chrome_debug_port = "9222"
user_data_dir = "C:/Selenium/ChromeData"

# Chrome을 원격 디버깅 모드로 실행
subprocess.Popen([chrome_path, f'--remote-debugging-port={chrome_debug_port}', f'--user-data-dir={user_data_dir}'])

# Selenium 옵션 설정
options = Options()
options.add_experimental_option("debuggerAddress", f"127.0.0.1:{chrome_debug_port}")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 구글 로그인 페이지로 이동
driver.get('https://accounts.google.com/signin')

# 수동으로 로그인 진행
print("Please log in manually and then press Enter here...")
input("Press Enter after you have logged in...")  # 터미널에서 엔터를 누름

# 로그인된 쿠키 저장
cookies = driver.get_cookies()
with open('cookies.pkl', 'wb') as file:
    pickle.dump(cookies, file)

print("Cookies saved.")
driver.quit()

Please log in manually and then press Enter here...
Cookies saved.


## 크롤링 메인 코드

In [2]:
import subprocess
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import json
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import os
import signal

# Chrome 브라우저를 원격 디버깅 모드로 실행
chrome_path = "C:/Program Files/Google/Chrome/Application/chrome.exe"
chrome_debug_port = "9222"
user_data_dir = "C:/Selenium/ChromeData"

# Chrome을 원격 디버깅 모드로 실행
subprocess.Popen([chrome_path, f'--remote-debugging-port={chrome_debug_port}', f'--user-data-dir={user_data_dir}'])

# Selenium 옵션 설정
options = Options()
options.add_experimental_option("debuggerAddress", f"127.0.0.1:{chrome_debug_port}")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def wait_for_page_load(driver, timeout=50):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: d.execute_script('return document.readyState') == 'complete'
        )
        return True
    except:
        return False

def scroll_to_element(driver, element):
    """ Scrolls the page until the element is visible """
    driver.execute_script("arguments[0].scrollIntoView();", element)

def get_buttons(driver, button_selector, timeout=30):
    """ Retrieves buttons with a given selector and waits for them to be loaded """
    end_time = time.time() + timeout
    buttons = []
    while time.time() < end_time:
        buttons = driver.find_elements(By.CSS_SELECTOR, button_selector)
        if buttons:
            return buttons
        time.sleep(2)
    return buttons

try:
    # 페이지 접속
    driver.get('https://alan.est.ai/youtube?showHistoryMenu=false&showLeftSidebar=true')
    if not wait_for_page_load(driver):
        print("페이지 로딩에 실패했습니다.")
        driver.quit()
        exit()

    driver.maximize_window()
    time.sleep(3)

    # 데이터 저장 리스트 및 실패한 URL 저장 리스트
    all_data = {}
    failed_urls = []

    # URL 리스트 경로
    path_list_file = r'C:\Users\wjsgh\Downloads\workspace\hacker_th6\team-project\src\Crawling\Data_Youtube URL\Dog\강형욱유튜브url모음\기초교육.csv'

    df = pd.read_csv(path_list_file)
   
    # url 몇개까지 할건지 개수지정. 앨런=하루 30개 제한, ex) [:30], [31:60], [61:90]...
    path_list = df['링크'].tolist()[:30] 

    # 각 URL에 대해 반복 작업 수행
    for url in path_list:
        print(f"처리 중인 URL: {url}")
        driver.get('https://alan.est.ai/youtube?showHistoryMenu=false&showLeftSidebar=true')
        if not wait_for_page_load(driver):
            print(f"URL {url}에 대한 페이지 로딩에 실패했습니다.")
            failed_urls.append(url)
            continue

        try:
            textarea = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'textarea[placeholder*="요약할 유튜브 링크를 입력해주세요"]'))
            )
            textarea.clear()
            textarea.send_keys(url)
            textarea.send_keys(Keys.RETURN)
            time.sleep(3)

            # 2. 로딩 대기
            WebDriverWait(driver, 180).until(
                EC.presence_of_element_located((By.XPATH, '//p[contains(text(), "영상 요약을 완료했습니다!")]'))
            )
            print(f"요약 완료 메시지가 표시되었습니다: {url}")
            time.sleep(10)

            # 3. 타이틀 추출
            titles = driver.find_elements(By.CSS_SELECTOR, 'div._title_1skjv_47')
            print(f"타이틀 개수: {len(titles)}")
            time.sleep(5)

            # 4. 버튼 로딩 및 클릭
            buttons = get_buttons(driver, 'button._btn_arrow_1skjv_57')
            if not buttons:
                print("버튼이 로딩되지 않았습니다.")
                continue
            print(f"버튼 개수: {len(buttons)}")
            time.sleep(5)

            url_data = {
                'URL': url,
                'SUMMARIES': []
            }

            for i, button in enumerate(buttons):
                try:
                    scroll_to_element(driver, button)
                    ActionChains(driver).move_to_element(button).click().perform()
                    print(f"버튼 클릭됨: {i}")
                    time.sleep(3)
                except Exception as e:
                    print(f"버튼 클릭 오류: {e}")
                    continue
            time.sleep(5)

            # 5. '자세히' 버튼 로딩 및 클릭
            detail_buttons = get_buttons(driver, 'button._btn_detail_rtro1_37')
            if not detail_buttons:
                print("'자세히' 버튼이 로딩되지 않았습니다.")
                continue
            print(f"'자세히' 버튼 개수: {len(detail_buttons)}")
    
            for i, detail_button in enumerate(detail_buttons):
                try:
                    # 스크롤하여 버튼을 클릭 가능하게 함
                    scroll_to_element(driver, detail_button)
                    ActionChains(driver).move_to_element(detail_button).click().perform()
                    print(f"'자세히' 버튼 클릭됨: {i}")
                    time.sleep(3)  # 버튼 클릭 후 로딩 시간 대기

                    # 타이틀 추출
                    summary_title = titles[i].text if i < len(titles) else f"Summary {i+1}"
                    print(f"타이틀 추출됨: {summary_title}")

                    # 간단 텍스트 모음 추출
                    simple_text_elements = driver.find_elements(By.XPATH, '//ul[@class="_ul_tnadh_1"]/li/span/p[@class="_p_qdrcw_1"]')
                    simple_text = "\n".join([element.text for element in simple_text_elements])
                    print(f"간단 텍스트 추출됨: {simple_text}")

                    # 자세히 텍스트 모음 추출
                    detail_text_elements = driver.find_elements(By.XPATH, '//div[@class="_detail_rtro1_31"]/p[@class="_p_qdrcw_1"]')
                    detail_text = "\n".join([element.text for element in detail_text_elements])
                    print(f"자세히 텍스트 추출됨: {detail_text}")

                    # 추출한 데이터를 저장
                    url_data['SUMMARIES'].append({
                        'SUMMARY_TITLE': summary_title,
                        'SIMPLE_TEXT': simple_text,
                        'DETAIL_TEXT': detail_text
                    })
                except Exception as e:
                    print(f"'자세히' 버튼 클릭 오류: {e}")

            all_data[url] = url_data

        except Exception as e:
            print(f"Error processing URL {url}: {e}")
            failed_urls.append(url)
            continue

    # JSON 파일로 저장
    try:
        with open('extracted_data.json', 'w', encoding='utf-8') as f:
            json.dump(all_data, f, ensure_ascii=False, indent=4)
        print("데이터가 extracted_data.json 파일에 저장되었습니다.")
    except Exception as e:
        print(f"JSON 파일 저장 중 오류 발생: {e}")

    # 실패한 URL 저장 (선택 사항)
    if failed_urls:
        with open('failed_urls.txt', 'w') as f:
            for url in failed_urls:
                f.write(url + '\n')

finally:
    # 드라이버 종료
    driver.quit()

    os.kill(int(chrome_debug_port), signal.SIGTERM)

처리 중인 URL: https://www.youtube.com/watch?v=kRsYuNPOzos&list=PLVh3TM0B0WtntFLpB0oH4I8zAxbO-2xLK&index=1&pp=iAQB
요약 완료 메시지가 표시되었습니다: https://www.youtube.com/watch?v=kRsYuNPOzos&list=PLVh3TM0B0WtntFLpB0oH4I8zAxbO-2xLK&index=1&pp=iAQB
타이틀 개수: 11
버튼 개수: 11
버튼 클릭됨: 0
버튼 클릭됨: 1
버튼 클릭됨: 2
버튼 클릭됨: 3
버튼 클릭됨: 4
버튼 클릭됨: 5
버튼 클릭됨: 6
버튼 클릭됨: 7
버튼 클릭됨: 8
버튼 클릭됨: 9
버튼 클릭됨: 10
'자세히' 버튼 개수: 11
'자세히' 버튼 클릭됨: 0
타이틀 추출됨: 강아지의 주인 문제와 환경
간단 텍스트 추출됨: 강아지의 주인이 명확하지 않다는 문제를 다루고 있습니다
거주 환경에 따라 반려견과 살아가는 모습이 달라진다는 점을 강조합니다
강형욱 훈련사가 다양한 반려인과 삶의 이야기를 나누기 위해 방문합니다
강아지 바다가 사무실에서 너무 활발하게 행동하는 문제를 다룹니다
강아지가 사무실에서 소변을 보는 문제도 언급됩니다
강형욱 훈련사는 줄을 잡고 밖으로 나가도록 유도합니다
강아지 바다가 훈련소에서 2개월을 보내고 왔으나 효과가 오래가지 않음
사무실에 마당이 있어 강아지를 키우기 적합하다고 생각했으나 활발한 성격이 문제
훈련소 경험 후에도 사무실에서의 문제가 지속됨
강아지 바다의 주인이 명확하지 않음
사무실 숙소에서 여러 직원들이 돌아가며 강아지를 돌봄
이로 인해 강아지의 안정감이 부족하고 문제가 발생함
강아지 바다가 사무실에서 돌아다닐 수 있는 공간이 있음
강아지가 카페테리아에서 지내는 것은 적합하지 않음
강아지 주인이 명확하지 않으면 문제가 발생함
강아지를 공공 킥보드처럼 키우면 안 된다는 점을 강조함
강아지와의 상호작용에서 갈증이 있을 때 물고 당김
강아지를 예뻐해주고 상호작용하는 방법을 지도함
강형욱 훈련사가 